In [1]:
import sys
import pickle
import docplex.mp

In [2]:
d1 = 25
d2 = 35
with open("cost.txt", "rb") as fp:   # Unpickling
    Costs = pickle.load(fp)

In [3]:
R1 = range(1,d1)
R2 = range(1,d2);

dim  = range(1,d1*d2+1)

In [4]:
# first import the Model class from docplex.mp
from docplex.mp.model import Model

m = Model(name='benders', log_output=True)

In [5]:
X = m.continuous_var_dict([(i,j) for i in R2 for j in R1])
Y = m.integer_var_dict(R1, 0, 1)


bendersPartition = {(i,j) : i for i in R2 for j in R1}

In [7]:
m.minimize( m.sum( Costs[i][j]*X[i,j] for i in R2 for j in R1) + sum(Y[i] for i in R1) )

m.add_constraints( m.sum( X[i,j] for j in R1) ==1 for i in R2)
    
m.add_constraints( X[i,j] - Y[j] <= 0 for i in R2 for j in R1)

m.print_information()

Model: benders
 - number of variables: 840
   - binary=0, integer=24, continuous=816
 - number of constraints: 1700
   - linear=1700
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [8]:
m.parameters.benders.strategy = 3  # <-1, OFF>, <0, AUTO>, <1, USER>, <2, WORKERS>, <3, FULL>

In [9]:
msol = m.solve(clean_before_solve=True)
m.report()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Benders_Strategy                        3
Found incumbent of value 494.000000 after 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 850 rows and 0 columns.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.64 ticks)
Tried aggregator 1 time.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.93 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0        0.0000            494.0000    Benders: 2        0  100.00%
      0     0        4.0000            494.0000    Benders: 7        2   99.19%
      0     0       12.8125            494.0000   

In [10]:
m.parameters.benders.strategy = 1
for i in R2:
    for j in R1:
        X[i,j].benders_annotation =  i%2

In [11]:
msol = m.solve(clean_before_solve=True)
m.report()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Benders_Strategy                        1
Found incumbent of value 494.000000 after 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 850 rows and 0 columns.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.64 ticks)
Tried aggregator 1 time.
Reduced MIP has 850 rows, 840 columns, and 2448 nonzeros.
Reduced MIP has 24 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.93 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0       24.0000            494.0000    Benders: 1       21   95.14%
      0     0       26.4615            494.0000    Benders: 1       25   94.64%
      0     0       30.7500            494.0000   